In [4]:
import os
import shutil
import pandas as pd
import random

In [5]:
SOURCE_DATASET_DIR = "../content/processed_data" 
SEED_OUTPUT_DIR = "../content/seed_images_annotated"             
CSV_OUTPUT_PATH = "../content/concepts_to_annotate.csv"
IMAGES_PER_CLASS = 10

In [6]:
def create_seed_kit():
    if os.path.exists(SEED_OUTPUT_DIR):
        shutil.rmtree(SEED_OUTPUT_DIR)
    os.makedirs(SEED_OUTPUT_DIR)

    classes = ["Hemorrhagic", "Ischemic", "Tumor"]
    data_rows = []

    print(f"Selecting {IMAGES_PER_CLASS} random *processed* images per class...")

    for cls in classes:
        class_path = os.path.join(SOURCE_DATASET_DIR, cls)
        # Only pick the 'real' ones for annotation, not the augmented ones
        images = [f for f in os.listdir(class_path) if f.startswith("real_") and f.endswith('.jpg')]
        
        # Guard clause if we have fewer than 10 images
        count = min(len(images), IMAGES_PER_CLASS)
        selected_images = random.sample(images, count)

        for img_name in selected_images:
            # Copy image to seed folder
            src = os.path.join(class_path, img_name)
            dst_name = f"{cls}_{img_name}" 
            dst = os.path.join(SEED_OUTPUT_DIR, dst_name)
            shutil.copy(src, dst)

            # Add row to CSV data
            data_rows.append({
                "filename": dst_name,
                "class_label": cls,
                "has_hyperdensity": 0, # Bright white spot?
                "has_hypodensity": 0,  # Dark spot?
                "has_mass_effect": 0,  # Brain pushed/squashed?
                "has_midline_shift": 0 # Center line bent?
            })

    # Save CSV
    df = pd.DataFrame(data_rows)
    df.to_csv(CSV_OUTPUT_PATH, index=False)
    
    print(f"\nSUCCESS! Seed Kit Generated.")
    print(f"1. Download folder: {SEED_OUTPUT_DIR}")
    print(f"2. Download CSV: {CSV_OUTPUT_PATH}")
    print(f"3. Action: Open CSV, look at images, change 0 to 1 where true. Save.")

create_seed_kit()

Selecting 10 random *processed* images per class...

SUCCESS! Seed Kit Generated.
1. Download folder: ../content/seed_images_annotated
2. Download CSV: ../content/concepts_to_annotate.csv
3. Action: Open CSV, look at images, change 0 to 1 where true. Save.


In [7]:
import pandas as pd
import random

# --- CONFIG ---
INPUT_CSV = "../content/concepts_to_annotate.csv"
OUTPUT_CSV = "../content/annotated_concepts_filled.csv"
# --------------

def auto_annotate():
    df = pd.read_csv(INPUT_CSV)
    
    # Define "Rules of Thumb" for each class
    # 1 = True (Always), 0.3 = 30% chance (Random)
    
    for index, row in df.iterrows():
        cls = row['class_label']
        
        if cls == "Hemorrhagic":
            # Bleeds are bright (Hyperdense)
            df.at[index, 'has_hyperdensity'] = 1
            df.at[index, 'has_hypodensity'] = 0
            # Large bleeds push the brain
            df.at[index, 'has_mass_effect'] = 1 if random.random() < 0.4 else 0
            df.at[index, 'has_midline_shift'] = 1 if random.random() < 0.2 else 0
            
        elif cls == "Ischemic":
            # Dead tissue is dark (Hypodense)
            df.at[index, 'has_hyperdensity'] = 0
            df.at[index, 'has_hypodensity'] = 1
            # Usually less swelling than tumors
            df.at[index, 'has_mass_effect'] = 1 if random.random() < 0.1 else 0
            df.at[index, 'has_midline_shift'] = 0
            
        elif cls == "Tumor":
            # Tumors can be anything, but usually have Mass Effect
            df.at[index, 'has_hyperdensity'] = 1 if random.random() < 0.6 else 0
            df.at[index, 'has_hypodensity'] = 1 if random.random() < 0.4 else 0
            df.at[index, 'has_mass_effect'] = 1 # Almost always pushes tissue
            df.at[index, 'has_midline_shift'] = 1 if random.random() < 0.5 else 0

    df.to_csv(OUTPUT_CSV, index=False)
    print("Synthetic Annotation Complete!")
    print(df.head(10))

auto_annotate()

Synthetic Annotation Complete!
                   filename  class_label  has_hyperdensity  has_hypodensity  \
0  Hemorrhagic_real_340.jpg  Hemorrhagic                 1                0   
1  Hemorrhagic_real_244.jpg  Hemorrhagic                 1                0   
2   Hemorrhagic_real_98.jpg  Hemorrhagic                 1                0   
3   Hemorrhagic_real_26.jpg  Hemorrhagic                 1                0   
4  Hemorrhagic_real_272.jpg  Hemorrhagic                 1                0   
5  Hemorrhagic_real_322.jpg  Hemorrhagic                 1                0   
6   Hemorrhagic_real_20.jpg  Hemorrhagic                 1                0   
7  Hemorrhagic_real_190.jpg  Hemorrhagic                 1                0   
8  Hemorrhagic_real_170.jpg  Hemorrhagic                 1                0   
9  Hemorrhagic_real_138.jpg  Hemorrhagic                 1                0   

   has_mass_effect  has_midline_shift  
0                0                  1  
1                0 